<a href="https://colab.research.google.com/github/Vulpardi/Projeto_Alura/blob/main/Briefing_Design.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalando o SDK do Google

#Criador de Briefing para designers

In [ ]:
import pandas as pd

In [ ]:
!pip install -q -U google-generativeai

In [ ]:
#Configurações iniciais
import google.generativeai as genai

GOOGLE_API_KEY="GOOGLE_API_KEY"
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
#Listando os modelos disponíveis
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


Como o objetivo é criar briefings criativos e únicos que causem impacto no portfólio do designer, as configs foram alteradas para melhor performance da alucinação e os parâmetros de segurança foram alterados para prover melhores resultados.

In [42]:
#Criando parâmetros de Alucinação
generation_config = {
  "candidate_count": 1,
  "temperature": 1,
  "max_output_tokens" :1536,
  "top_k" : 90,
}

In [38]:
#Alterando parâmetros de segurança
safety_settings={
    'HATE': 'BLOCK_LOW_AND_ABOVE',
    'HARASSMENT': 'BLOCK_LOW_AND_ABOVE',
    'SEXUAL' : 'BLOCK_NONE',
    'DANGEROUS' : 'BLOCK_NONE'
    }

In [43]:
model = genai.GenerativeModel(model_name='gemini-1.0-pro',
                                  generation_config=generation_config,
                                  safety_settings=safety_settings,)

# A partir dessa etapa, paramos de estabelecer as bases e começamos a elaborar o Prompt.

*O Prompt final vai utilizar técnicas de Chain-Of-Thought para, através de uma amostra, conseguir criar briefings claros e impactantes para o portfólio.*

A sequência para obter essa tabela foi a seguinte:

1.   Treinar uma IA Generativa para elaborar bons briefings
2.   Coletar uma quantidade suficiente para que surjam alguns padrões
3.   Gerar no Gemini, através do Structured Point, uma lógica mais complexa para que mais amostras interessantes surjam
4.  Editar as amostras recebidas reforçando seus pontos fortes e apagando os pontos fracos (/content/Briefings criativos.xlsx)
5.  Requisitar que o próprio Gemini analisasse e criasse uma estrutura lógica interna para ser replicado (prompt +=)
6.  Fazer as conexões dentro do Python para que fosse possível obter resultados satisfatórios integrando a própria estrutura de treinamento dentro do Gemini.


In [40]:
df = pd.read_excel("/content/Briefings criativos.xlsx")
df.head()

,Comando:,Segmento,Marca,contexto,Briefing,Objetivo,Persona,Desafio Extra
0,Crie um briefing criativo com o tema ALEATÓRIO...,Role-Playing Game (RPG),RealmCraft,RealmCraft é uma comunidade online dedicada ao...,Desenvolva uma identidade visual completa para...,A marca busca atrair uma comunidade vibrante e...,Lucas é um jovem apaixonado por jogos de RPG e...,Você não pode utilizar imagem de nenhuma criat...
1,Crie um briefing criativo com o tema LIMPEZA c...,Produtos de Limpeza,VitalEco,VitalEco é uma startup que traz uma abordagem ...,Desenvolva uma identidade visual completa para...,A marca busca atrair consumidores que valoriza...,"Joana, uma mãe da classe A preocupada com a sa...",Utilize apenas cores dentro do espectro de ver...
2,Crie um briefing criativo com o tema ALEATÓRIO...,Moda,SteamCraft,SteamCraft é uma marca de acessórios de luxo c...,Desenvolva uma identidade visual completa para...,A marca busca atrair consumidores que estão ca...,Carla tem 25 anos e sente necessidade de se ex...,Crie um acessório único e exclusivo da marca e...
3,Crie um briefing criativo com o tema ALEATÓRIO...,Energético,VitalBoost,VitalBoost é uma marca de energéticos saudáveis.,Desenvolva uma identidade visual completa além...,A marca busca transmitir a sensação de frescor...,Pedro tem 19 anos e busca um estilo de vida sa...,Utilize uma identidade visual minimalista e ut...
4,Crie um briefing criativo com o tema COSMÉTICO...,Cosméticos,Glow Essentials,Glow Essentials é uma marca criada por uma blo...,Desenvolva o design de embalagem para uma nova...,A marca busca atrair consumidoras que buscam p...,Julie é uma mulher de 35 anos de classe média ...,Crie embalagens com uma experiência tátil únic...


A ultima linha do código deve ser alterada para os seguintes parâmetros:

```
PROMPT: Crie um briefing criativo com o tema ALEATÓRIO com foco em ALEATÓRIO
```


O tema vai definir de qual segmento irá se tratar o Briefing e o Foco irá definir quais as demandas deverão ser entregues, Então um tema de Saúde com Foco em Social Media  pode fornecer um resultado como: "Criar 5 posts para um hospital"



In [41]:
from re import X

tema = input("Qual o tema do briefing? ")
foco = input("Qual o foco do briefing? ")

prompt = ""

for i in range(df.shape[0]):
  linha = df.iloc[i]
  prompt += f"""
  PROMPT: {linha["Comando:"]}
  Segmento: {linha["Segmento"]}
  Persona: {linha["Persona"]}
  Objetivo: {linha["Objetivo"]}
  Contexto: {linha["contexto"]}
  Briefing: {linha["Briefing"]}
  Desafio Extra: {linha["Desafio Extra"]}
  Marca: {linha["Marca"]}

  """
prompt += f"""
Seja um designer sênior e utilize as informações para construir ideias de briefings perfeitos para designer iniciantes que precisam melhorar seus portfólios

Os dados abaixo reúne diversas ideias de bons briefings para designer gráficos e apresenta dados e desafios com o intuito de melhorar o portfólio desses designers. Ele é composto por 8 campos, desses, o "comando" é o único que é realizado um "input" pelo usuário, os outros são apenas decorrências desse INPUT que possui duas variáveis, o TEMA que é de escolha livre do usuário e o FOCO que pode ser selecionado entre ALEATÓRIO, REDES SOCIAIS, PAPELARIA, EMBALAGEM. Quero que você analise essa planilha e me apresente uma ideia de Briefing:

PROMPT: Comando que será fornecido.
Segmento: Definido primariamente pelo TEMA, serve como base para o desenvolvimento dos demais elementos.
Persona: Entender as características, necessidades e desejos do público é crucial para criar uma identidade visual e peças de comunicação eficazes.
Objetivo: Definir o que a marca busca alcançar com a identidade visual e as peças de comunicação, norteando as decisões criativas.
Contexto: Uma vez compreendido o Segmento, Persona e Objetivo, a descrição da marca se torna mais rica e precisa.
Briefing: Com os elementos anteriores definidos, o briefing principal se torna claro e conciso, descrevendo as peças a serem criadas e os objetivos específicos de cada uma.
Desafio Extra: Elabore restrições criativas ou sujira direcionamentos inusitados.
Marca: O nome da marca pode ser definido após a criação do contexto, garantindo um nome que reflita a proposta de valor e o público-alvo.
Peças de comunicação (Posts, Banners, Papelaria, Embalagem): Com todas as informações anteriores em mente, o designer pode criar uma quantidade de peças de comunicação visual que atendam aos objetivos da marca e ressoem com o público-alvo.

Dadas essa estrutura eu posso elaborar o exemplo a seguir:

PROMPT: Crie um briefing criativo com o tema JOGOS com foco em ALEATÓRIO
Segmento: Videogames
Persona: Paulo é um jovem de 25 anos e um ávido fã de jogos de corrida. Ele busca jogos que ofereçam gráficos impressionantes, física realista e uma jogabilidade envolvente que teste suas habilidades de condução. Paulo valoriza a inovação, a qualidade e a comunidade de jogadores que compartilham sua paixão por corridas virtuais.
Objetivo:  A marca procura estabelecer-se como líder na indústria de jogos de corrida, atrair jogadores apaixonados por competições de alta velocidade e experiências de condução realistas.
Contexto: A CodeMasters é uma marca reconhecida no desenvolvimento e publicação de jogos de corrida de alta qualidade, que proporcionam aos jogadores experiências emocionantes e realistas.
Briefing: Desenvolva uma identidade visual completa para a CodeMasters. A identidade visual deve transmitir a natureza inovadora, envolvente e competitiva dos videogames de corrida.
Desafio Extra: Incorporar elementos do automobilismo na identidade visual e evitar o uso de imagens de carros genéricos.
Marca: CodeMasters
Quantidade de peças: 1 cartães de visita, 1 papel timbrado, 1 caneca e 2 Posts

PROMPT: Crie um briefing criativo com o tema {tema} com foco em {foco}
"""
print(prompt)

Qual o tema do briefing? ALEATÓRIO
Qual o foco do briefing? SOCIAL MEDIA

  PROMPT: Crie um briefing criativo com o tema ALEATÓRIO com foco em REDES SOCIAIS
  Segmento: Role-Playing Game (RPG)
  Persona: Lucas é um jovem apaixonado por jogos de RPG e universos de fantasia desde a infância. Ele busca comunidades online onde possa se conectar com outros fãs, compartilhar suas experiências e encontrar inspiração para suas próprias aventuras. Lucas valoriza conteúdo visualmente impactante e criativo, que o transporte para mundos mágicos e emocionantes.
  Objetivo: A marca busca atrair uma comunidade vibrante e engajada, então o conteúdo deve focar em gerar engajamento nas postagens e mensagens no direct da empresa.
  Contexto: RealmCraft é uma comunidade online dedicada aos entusiastas de RPG e jogos de fantasia.
  Briefing: Desenvolva uma identidade visual completa para RealmCraft, com 4 posts e 1 Banner digital. A identidade visual deve transmitir a magia, o mistério e a aventura dos RPG

# E esse é o resultado final...

In [44]:
response = model.generate_content(prompt)
print(response.text)

Segmento: Cozinha Gourmet
Persona: Rafaela é uma cozinheira amadora apaixonada por experimentar novos sabores e receitas. Ela busca equipamentos de cozinha de alta qualidade que possam ajudá-la a elevar suas habilidades culinárias. Rafaela valoriza a funcionalidade, o design elegante e a durabilidade dos utensílios de cozinha, e está sempre procurando por marcas que compartilhem sua paixão pela gastronomia.
Objetivo: A marca busca atrair cozinheiros amadores e entusiastas que buscam utensílios de cozinha confiáveis e inspiradores para aprimorar sua jornada culinária.
Contexto: Chef's Haven é uma marca dedicada ao fornecimento de utensílios de cozinha premium para cozinheiros amadores e profissionais. Seus produtos combinam qualidade excepcional com design inovador, atendendo às necessidades e aspirações dos apaixonados por culinária.
Briefing: Desenvolva uma identidade visual completa e uma campanha para as redes sociais da Chef's Haven. A identidade visual deve transmitir a qualidade,